In [1]:
import numpy as np

In [655]:
# todo: debug changes: Arguments are immutable in R
class e_divisive:
    def __init__(self):
        print('case initialized')
        
    def divisive(self,Z, k =None, p0=.05, m = 30, R=199, alpha=2):
        self.Z = Z
        self.p0 = p0
        self.min_size = m
        self.R = R
        self.alpha = alpha
        self.D = self.create_distance_matrix(Z)
        self.length = len(Z)
        self.energy = np.zeros((self.length,2))
        if k is None:
            self.k = self.length
        else:
            self.R = 0
        ret = pvals = permutations = None
        changes = [0,self.length]
        k_hat = 1 
        con = None
        while self.k > 0:
            # tmp [0, 149, [0, 150, 50], 3647.021671525799]
            # results [0.385, 199]
            tmp = self.split(changes, self.D, self.min_size, False, self.energy)
            i = tmp[0]
            j = tmp[1]
            new_changes = tmp[2]
            estat = tmp[3]
            con = new_changes[-1]
            if con == -1:
                break
            result = self.sig_test(self.D, self.R, changes, self.min_size, estat,
                                  self.energy)
            pval = result[0]
            permutations = [permutations, result[1]]
            pvals = [pvals, pval]
            if pval > self.p0:
                break
            changes = new_changes
            k_hat += 1
            self.k -= 1
        tmp = sorted(changes)
        return tmp
            
    def create_distance_matrix(self, Z):
        length = len(Z)
        matrix = np.zeros((length, length))
        for i in range(length):
            for j in range(length):
                matrix[i][j] = np.abs(Z[i] - Z[j])**self.alpha
        return matrix   

    def sig_test(self, D, R, changes, min_size, estat, energy):
        if R == 0:
            return 0
        over = 0
        for i in range(R):
            D1 = self.perm_cluster(D, changes)
            tmp = self.split(changes = changes, D =D1, min_size = min_size, \
                             for_sim = True, energy=energy)
            if tmp[3]>=estat:
                over += 1
        p_val = (1+over)/(R+1) # todo: might be problematic and -1
        return [p_val, R]
    
    def perm_cluster(self, D, changes):
        D1 = D.copy()
        changes1 = changes.copy()
        changes1 = sorted(changes1)
        K = len(changes1) - 1
        for i in range(K):
            u = np.arange(changes1[i],changes1[i+1])
            np.random.shuffle(u)
            self.u = u
            mesh = np.meshgrid(u,u,indexing='ij')
            D1[changes1[i]:changes1[i+1],changes1[i]:changes1[i+1]]=D1[mesh[0],mesh[1]]
        return D1
            
    def split_pointC(self, start, end, D, min_size):
        s = start
        best = [-1, -np.inf]
        num_points = end - start + 1
        tau = min_size
        keppa = min_size << 1
        cut1 = D[s:tau,s:tau] # notice the difference with Rc++
        cut2 = D[tau:keppa,tau:keppa]
        cut3 = D[s:tau,tau:keppa]
        A = np.sum(cut1)/2
        B1 = np.sum(cut2)/2
        AB1 = np.sum(cut3)
        tmp = 2*AB1/((keppa-tau)*tau) - 2*B1/((keppa-tau-1)*(keppa-tau)) - \
                                            2*A/((tau-1)*tau)
        tmp *= (tau*(keppa-1)/keppa)
        if tmp > best[1]:
            best[0]=tau+start
            best[1] = tmp
        keppa += 1

        B = np.ones(num_points+1)*B1 # memorization, 
        AB = np.ones(num_points+1)*AB1
        for i in range(keppa, num_points):
            B[i] = B[i-1]+np.sum(D[i,tau:i-1])# mind the interval
            AB[i] = AB[i-1]+np.sum(D[i,0:tau])
            tmp = 2*AB[i]/((i-tau)*tau) - 2*B[i]/((i-tau-1)*(i-tau))-2*A/(tau*(tau-1))
            tmp *= (tau*(i-tau)/i)
            if tmp > best[1]:
                best[0] = tau + start
                best[1] = tmp
        tau += 1

        for i in range(tau,end):
            keppa = i + min_size
            if keppa > num_points:
                break
            addA = np.sum(D[i-1,0:i-1])
            A+=addA
            addB = np.sum(D[i-1,i:keppa-1])
            for j in range(keppa,num_points):
                addB += D[i-1,j-1]
                B[j]-=addB
                AB[j]+=(addB -addA)
                tmp = 2*AB[j]/((j-i)*i)-2*B[j]/((j-i-1)*(j-i))-2*A/((i-1)*i)
                tmp *= (i*(j-i)/j)
                if tmp > best[1]:
                    best[0] = i+start
                    best[1] = tmp
        return best

    def splitPoint(self, start, end, D, min_size):
        if end - start + 1<2*min_size:
            return [-1, -np.inf]
        D1 = D[start:end+1, start:end+1]
        return split_pointC(start=start, end=end, D=D1, min_size=min_size)

    def split(self, changes, D, min_size, for_sim, energy):
        changes1 = changes.copy()
        splits = sorted(changes1)
        best = [-1,-np.inf]
        ii = jj = -1
        if for_sim:
            for i in range(1,len(splits)):
                tmp = splitPoint(splits[i-1],splits[i]-1,D,min_size)
                if tmp[1]>best[1]:
                    ii = splits[i-1]
                    jj = splits[i]-1
                    best = tmp
            new_changes = changes1.copy()
            new_changes.append(best[0])
            return [ii,jj,new_changes,best[1]]
        else:
            for i in range(1, len(splits)):
                if energy[splits[i-1],0] != 0:
                    tmp = energy[splits[i-1],]
                else:
                    tmp = splitPoint(splits[i-1],splits[i]-1,D,min_size)
                    energy[splits[i-1],0] = tmp[0]
                    energy[splits[i-1],1] = tmp[1]
                if tmp[1]>best[1]:
                    ii = splits[i-1]
                    jj = splits[i]-1
                    best = tmp
        new_changes = changes1.copy()
        new_changes.append(best[0])
        energy[ii,0] = 0
        energy[jj,1] = 0
        return [ii,jj,new_changes,best[1]]

In [662]:
l1 = 50
l2 = 50
l3 = 50
energy = - np.ones((l1+l2+l3,2))
x1 = np.random.normal(0,.1,l1)
x2 = np.random.normal(1,.1,l2)
x3 = np.random.normal(2,.1,l3)
x = np.r_[x1,x2,x3]
D = np.zeros((l1+l2+l3,l1+l2+l3))
energy = np.zeros((150,2))
for i in range(l1+l2+l3):
    for j in range(l1+l2+l3):
        D[i][j] = np.abs(x[i]-x[j])**2

In [663]:
ed = e_divisive()

case initialized


In [664]:
tmp = ed.divisive(Z=x,p0=.05,m=30)

In [665]:
tmp

[0, 50, 100, 150]